In [231]:
import pandas as pd
import numpy as np
from datetime import datetime


In [232]:

#CARGAR EL ARCHIVO CON LOS DATOS DE LA AGENCIA COMPETITIVA
ruta_agencia_competitiva = r"C:\Users\sgarcial\Documents\ETB\Agencia competitiva\Sabanas temporales\DatosAgencia_Enero.xlsx"
df = pd.read_excel(
    ruta_agencia_competitiva,
    usecols=['Telefono', 'Nivel_1', 'Franja','NombreServicio', 'Fuente', 'Venta',  'Nivel3'],
    sheet_name='Datos'
)

#CARGAR ARCHIVO CRITERIOS DE LA AGENCIA 14
ruta_criterios = r"C:\Users\sgarcial\Documents\ETB\Agencia competitiva\Sabanas temporales\Analisis reinyecciones 960020 Competitiva (1).xlsx"
df_criterios = pd.read_excel(
    ruta_criterios,
    sheet_name="Criterios",
    header=None,
)

df_criterios_telefono = df_criterios.copy()

In [233]:
print(df.shape)
df.head()

(1550, 7)


,Franja,Telefono,Nivel_1,NombreServicio,Fuente,Venta,Nivel3
0,2230,3204126250,ILOCALIZABLES,C2C_2,N/D,0,Contestador
1,2100,3057384654,NaN,reinyeccion whatsapps1,NaN,0,NaN
2,1530,3192677915,Atencion,WhatsappS1,CPWhatsappOctubre,0,NaN
3,1030,3204029362,EN PROCESO,WhatsappS1,NaN,0,CON OFERTAMIENTO
4,1430,3011955168,NO VENTA,C2C_2,CPCampañaHalloween,0,NO HAY COBERTURA OTRA CIUDAD


In [234]:
df_criterios.head()

,0,1,2,3,4,5,6,7,8
0,LineaIn018000,LLAMADA,NaN,TIPIFICACIONES CHAT,NaN,NaN,NaN,TIPIFICACIONES LLAMADA,NaN
1,WhatsappS1,CHAT,NaN,SIN OFERTAMIENTO,TENER EN CUENTA,NaN,NaN,SIN COBERTURA,NO TENER EN CUENTA
2,C2C_2,LLAMADA,NaN,DIRECCION REQUIERE ESCALAMIENTO,TENER EN CUENTA,NaN,NaN,Cuelga en espera,TENER EN CUENTA
3,Linea_Inbound_linkedin,LLAMADA,NaN,CON OFERTAMIENTO,TENER EN CUENTA,NaN,NaN,NO VOLVER A LLAMAR,NO TENER EN CUENTA
4,Movil In,LLAMADA,NaN,NO ACEPTA NO INDICA RAZON,TENER EN CUENTA,NaN,NaN,LLAMADA NO SERIA,NO TENER EN CUENTA


In [235]:
#APLICAR FILTROS A LAS COLUMNAS NIVEL1, NOMBRE SERVICIO Y VENTA

#FILTRO A LA COLUMNA NIVEL 1
valores_excluir_Nivel1 = [
    "ATENCIÓN", "ATENCION",
    "ATENCIÓN, CONTACTO NO UTIL", "ATENCION, CONTACTO NO UTIL",
    "ATENCIÓN, EN PROCESO", "ATENCION, EN PROCESO",
    "ATENCIÓN, NO VENTA", "ATENCION, NO VENTA",
    "ATENCION, SERVICIO AL CLIENTE", "ATENCION, SERVICIO AL CLIENTE",
    "ATENCIÓN, VENTA HOGARES", "ATENCION, VENTA HOGARES",
    "ATENCIÓN, VENTA MOVIL", "ATENCIÓN, VENTA MOVIL", 
    "SERVICIO AL CLIENTE",
    "VENTA",
    "VENTA FTTH + MÓVIL", "VENTA FTTH + MOVIL"
    "VENTA HOGARES",
    "VENTA MOVIL", "VENTA MÓVIL",
    "VENTA NEGOCIO",
    "VENTA NEGOCIOS"]
df["Nivel_1"] = df["Nivel_1"].astype(str).str.strip().str.upper()
df = df[~df["Nivel_1"].isin(valores_excluir_Nivel1)]

#FILTRO A LA COLUMNA NOMBRE SERVICIO
valores_excluir_servicio = [
    "AGENCIA ETB PD REINYECCION", "AGENCIA ETB PD REINYECCIÓN",
    "AGENCIA ETB PT",
    "AGENCIA ETB PU",
    "BBDD_IR",
    "REFERIDOS_C2C_2"]
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df = df[~df["NombreServicio"].isin(valores_excluir_servicio)]

#FILTRO A LA COLUMNA VENTA
df["Venta"] = pd.to_numeric(df["Venta"], errors="coerce")
df = df[df["Venta"] == 0]



In [236]:
print(df.shape)
df.head()

(958, 7)


,Franja,Telefono,Nivel_1,NombreServicio,Fuente,Venta,Nivel3
0,2230,3204126250,ILOCALIZABLES,C2C_2,N/D,0,Contestador
1,2100,3057384654,NAN,REINYECCION WHATSAPPS1,NaN,0,NaN
3,1030,3204029362,EN PROCESO,WHATSAPPS1,NaN,0,CON OFERTAMIENTO
4,1430,3011955168,NO VENTA,C2C_2,CPCampañaHalloween,0,NO HAY COBERTURA OTRA CIUDAD
5,1430,3196666013,NO VENTA,C2C_2,CPRemarketingHogares,0,CLIENTE YA CUENTA CON SERVICIOS ETB


In [237]:
#ELIMANR GUIONES Y ESPACIOS DE LA COLUMNA Telefono
df["Telefono"] = (
    df["Telefono"]
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace("-", "", regex=False)
)


#CREAR COLUMNA TELEFONO2
df["TELEFONO2"] = (
    df["Telefono"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)
    .str[-10:]
)

df = df.drop(columns=["Telefono"])

In [238]:
df.head()

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2
0,2230,ILOCALIZABLES,C2C_2,N/D,0,Contestador,3204126250
1,2100,NAN,REINYECCION WHATSAPPS1,NaN,0,NaN,3057384654
3,1030,EN PROCESO,WHATSAPPS1,NaN,0,CON OFERTAMIENTO,3204029362
4,1430,NO VENTA,C2C_2,CPCampañaHalloween,0,NO HAY COBERTURA OTRA CIUDAD,3011955168
5,1430,NO VENTA,C2C_2,CPRemarketingHogares,0,CLIENTE YA CUENTA CON SERVICIOS ETB,3196666013


In [239]:
#CREAR COLUMNA CATEGORIA, ANTERIORMENTE =BUSCARV(Nombre Servicio; &#39; Análisis de reinyección Hoja1&#39;!A:B; 2; 0)
df_criterios_categoria = df_criterios.copy()
df_criterios_categoria = df_criterios_categoria.iloc[:, [0, 1]]
df_criterios_categoria.columns = ["NombreServicio", "CATEGORIA"]
#df_criterios_categoria = df_criterios_categoria.dropna(subset=["NombreServicio"])
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df_criterios_categoria["NombreServicio"] = df_criterios_categoria["NombreServicio"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios_categoria,
    on="NombreServicio",
    how="left"
)

df2 = df.copy()

In [240]:
df.sample(5)

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA
441,1300,NO VENTA,C2C_2,ETBInternetEx,0,CLIENTE YA CUENTA CON SERVICIOS ETB,3177745040,LLAMADA
411,800,NAN,REINYECCION WHATSAPPS1,NaN,0,NaN,3502011190,CHAT
186,1800,ILOCALIZABLES,C2C_2,Hogares,0,CONTESTADOR,3242818501,LLAMADA
850,1130,ILOCALIZABLES,C2C_2,CPCampañaHalloween,0,CONTESTADOR,3186154716,LLAMADA
863,1730,NO VENTA,WHATSAPPS1,CPWhatsappOctubre,0,SIN COBERTURA,3177316002,CHAT


In [241]:
#CREAR COLUMNA FUENTE, ANTERIORMENTE =BUSCARV(Fuente; &#39; Análisis de reinyección Hoja3&#39;!G:K; 5; 0)

# CARGAR LA HOJA CON EL RANGO
df_fuente = pd.read_excel(
    ruta_criterios,
    sheet_name="ANALISIS EFECTIVIDAD",
    header=None,
    engine="openpyxl"
)

df_criterios_fuente_telefono = df_fuente.copy()


# G = 6, K = 10 → 6:11
df_fuente = df_fuente.iloc[:, 6:11]

df_fuente.columns = [
    "Fuente",     # Columna G → CLAVE
    "COL_H",
    "COL_I",
    "COL_J",
    "FUENTE"      # Columna K → RESULTADO BUSCARV
]


df["Fuente"] = df["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()


df = df.merge(
    df_fuente[["Fuente", "FUENTE"]],
    on="Fuente",
    how="left"
)

#df['FUENTE'] = df['FUENTE'].fillna('BAJA')


In [242]:
df.sample(10)

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA,FUENTE
165,2000,ILOCALIZABLES,C2C_2,CPPIEZASDISRUPTIVAS,0,CONTESTADOR,3132976375,LLAMADA,BAJA
40,1930,NO VENTA,C2C_2,CPCAMPAÑAHALLOWEEN,0,TRAM PENDIENTES (OTRO ALIADO),3132603148,LLAMADA,BAJA
263,1100,NO VENTA,C2C_2,CPCAMPAÑAHALLOWEEN,0,OFRECE APPS INCLUIDAS EN OFERTA POR UN AÑO (O...,3143478127,LLAMADA,BAJA
78,830,EN PROCESO,C2C_2,CPREMARKETINGHOGARES,0,CON OFERTAMIENTO,3144169781,LLAMADA,BAJA
741,1100,ILOCALIZABLES,C2C_2,HOGARES,0,Teléfono no existe,3238771514,LLAMADA,BAJA
768,2030,EN PROCESO,C2C_2,CPPIEZASDISRUPTIVAS,0,Programado,3118612277,LLAMADA,BAJA
542,1300,NO VENTA,C2C_2,CPREMARKETINGHOGARES,0,NO HAY COBERTURA OTRA CIUDAD,3206571402,LLAMADA,BAJA
175,300,EN PROCESO,C2C_2,CPPIEZASDISRUPTIVAS,0,Programado,3026050206,LLAMADA,BAJA
469,1200,EN PROCESO,C2C_2,CPCAMPAÑAHALLOWEEN,0,Llamada cortada,3008296220,LLAMADA,BAJA
924,1800,EN PROCESO,C2C_2,CPREMARKETINGHOGARES,0,TRAM PENDIENTES (PROV Q ATIENDE),3214792304,LLAMADA,BAJA


In [ ]:
#CREAR COLUMNA TIPIFICACION, ANTERIORMENTE =BUSCARV(Nivel_3; &#39;Análisis de reinyección Hoja1&#39;!D:F; 2; 0)
# D = 3, 3 = 5 → 3:5
df_criterios = df_criterios.iloc[:129, 3:5]

df_criterios.columns = [
    "Nivel3",        # Columna D → CLAVE
    "TIPIFICACION"]  # Columna E → RESULTADO BUSCARV

#df_criterios = df_criterios.dropna(subset=["Nivel3"])
df["Nivel3"] = df["Nivel3"].astype(str).str.strip().str.upper()
df_criterios["Nivel3"] = df_criterios["Nivel3"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)


In [244]:
df.sample(10)

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA,FUENTE,TIPIFICACION
5500,1830,VENTA HOGARES,C2C_2,CPCAMPAÑAHALLOWEEN,0,BA SINGLE,3197778602,LLAMADA,BAJA,NaN
3939,0,ILOCALIZABLES,C2C_2,CPCAMPAÑAHALLOWEEN,0,CONTESTADOR,6978782,LLAMADA,BAJA,NaN
2630,830,NAN,REINYECCION WHATSAPPS1,NAN,0,NAN,3102970031,CHAT,NaN,NaN
2526,1700,ILOCALIZABLES,C2C_2,CPPIEZASDISRUPTIVAS,0,CONTESTADOR,3006896139,LLAMADA,BAJA,NaN
6603,900,NAN,WHATSAPPS1,CPWHATSAPPOCTUBRE,0,NAN,3143894548,CHAT,BAJA,NaN
11051,200,NAN,C2C_2,CPCAMPAÑAHALLOWEEN,0,NAN,3024704229,LLAMADA,BAJA,NaN
8881,2200,NAN,C2C_2,CPCAMPAÑAHALLOWEEN,0,NAN,3043770999,LLAMADA,BAJA,NaN
5672,2130,NAN,WHATSAPPS1,NAN,0,NAN,3223755075,CHAT,NaN,NaN
11098,200,NAN,C2C_2,CPCAMPAÑAHALLOWEEN,0,NAN,3024704229,LLAMADA,BAJA,NaN
8166,600,EN PROCESO,C2C_2,CPCAMPAÑAHALLOWEEN,0,PROGRAMADO,3214521106,LLAMADA,BAJA,NaN


In [245]:
#SE CAMBIA EL SEET ID POR EL DE AGENCIA COMPETITIVA

#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "10osDxoOJTMcaNbfJfJ7x8AhCvpqzeCtCImRwQ2k0Sbg"
sheet_name = "CONSOLIDADO%20WPP"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"


# 1) Leer Google Sheets (✓ correcto)
df_CONSOLIDADO_WPP_agencia_competitiva = pd.read_csv(
    url,
    usecols=["CELULAR"]
)
df_CONSOLIDADO_WPP_agencia_competitiva = df_CONSOLIDADO_WPP_agencia_competitiva.rename(
    columns={df_CONSOLIDADO_WPP_agencia_competitiva.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)

# 2) Normalizar (agregar esta parte que faltaba)
df["TELEFONO2"] = df["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_WPP_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_WPP_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()  
)



df_CONSOLIDADO_WPP_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_WPP_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"]
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace("-", "", regex=False)
)

df_CONSOLIDADO_WPP_agencia_competitiva = df_CONSOLIDADO_WPP_agencia_competitiva.drop_duplicates(subset=["TELEFONOS_GOOGLE_SHEETS"])


# 3) Merge CORREGIDO (lookup como BUSCARV)
df = df.merge(
    df_CONSOLIDADO_WPP_agencia_competitiva.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS con los teléfonos encontrados
df["YAREINYECTADOS"] = df["TELEFONOS_GOOGLE_SHEETS"]
df.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)



In [246]:
print(df_CONSOLIDADO_WPP_agencia_competitiva.shape)
df_CONSOLIDADO_WPP_agencia_competitiva.head()

(29, 1)


,TELEFONOS_GOOGLE_SHEETS
0,3174433169
1,3026091855
2,3003013462
3,3105631001
4,3132473205


In [ ]:
df_chat_competitiva = df.copy()
df_chat_competitiva = df_chat_competitiva[df_chat_competitiva['CATEGORIA'] == 'CHAT']  # ← CRÍTICO
df_chat_competitiva['FUENTE'] = df_chat_competitiva['FUENTE'].fillna('BAJA')
df_chat_competitiva = df_chat_competitiva[df_chat_competitiva['TIPIFICACION'] == 'TENER EN CUENTA']
df_chat_competitiva = df_chat_competitiva[~df_chat_competitiva['YAREINYECTADOS'].apply(pd.to_numeric, errors='coerce').notna()]

df_chat_competitiva = df_chat_competitiva.drop_duplicates(subset=["TELEFONO2", "CATEGORIA"], keep="first")

df_chat_competitiva = df_chat_competitiva[['TELEFONO2', 'FUENTE']].sort_values(by='TELEFONO2')
df_chat_competitiva["TELEFONO2"] = pd.to_numeric(df_chat_competitiva["TELEFONO2"], errors="coerce")
df_chat_competitiva = df_chat_competitiva[(df_chat_competitiva["TELEFONO2"] >= 3000000000) & (df_chat_competitiva["TELEFONO2"] <= 3999999999)]

df_chat_competitiva = df_chat_competitiva.sort_values(by='TELEFONO2')
df_chat_competitiva.rename(columns={'TELEFONO2': 'TELEFONO'}, inplace=True)

In [ ]:
print(df_chat_competitiva.shape)
df_chat_competitiva

(30, 2)


,TELEFONO2,FUENTE
9760,3016919649,BAJA
7489,3027262251,BAJA
6092,3027914880,BAJA
6904,3028035665,BAJA
9774,3057375010,ALTA
5492,3102829880,BAJA
9722,3112918136,BAJA
5791,3118712213,BAJA
6407,3123664329,BAJA
7183,3132559996,BAJA


In [249]:
df_chat_competitiva.shape

(30, 2)

In [250]:
# Fecha: dd_mm_yyyy
fecha = datetime.now().strftime('%d_%m_%Y')

# CAMBIAR 0.1 → 01 (sin punto)
nombre_archivo = f"AYERautomatizado_0.1_{fecha}_CHAT_AGENCIA_COMPETITIVA.xlsx"

ruta_completa = f"C:/Users/sgarcial/Documents/ETB/Agencia competitiva/CHAT/{nombre_archivo}"

df_chat_competitiva.to_excel(ruta_completa, index=False)
print(f"✅ Guardado: {nombre_archivo}")

✅ Guardado: AYERautomatizado_0.1_08_01_2026_CHAT_AGENCIA_COMPETITIVA.xlsx


In [251]:
df2.head()

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA
0,2230,ILOCALIZABLES,C2C_2,N/D,0,Contestador,3204126250,LLAMADA
1,2100,NAN,REINYECCION WHATSAPPS1,NaN,0,NaN,3057384654,CHAT
2,1030,EN PROCESO,WHATSAPPS1,NaN,0,CON OFERTAMIENTO,3204029362,CHAT
3,1430,NO VENTA,C2C_2,CPCampañaHalloween,0,NO HAY COBERTURA OTRA CIUDAD,3011955168,LLAMADA
4,1430,NO VENTA,C2C_2,CPRemarketingHogares,0,CLIENTE YA CUENTA CON SERVICIOS ETB,3196666013,LLAMADA


In [252]:
#CREAR LAS COLUMNAS PARA CATEGORIA == TELEFONO

# 1) CREAR COLUMNA FUENTE CON LOS RANGOS DE TELEFONO
df_fuente = df_criterios_fuente_telefono.iloc[:121, 11:16]  # L:P, filas 32-152 ✓
df_fuente.columns = ["Fuente", "COL_M", "COL_N", "COL_O", "FUENTE"]  # P = FUENTE ✓
df2["Fuente"] = df2["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()
df2 = df2.merge(df_fuente[["Fuente", "FUENTE"]], on="Fuente", how="left")


In [253]:
df2.head()

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA,FUENTE
0,2230,ILOCALIZABLES,C2C_2,N/D,0,Contestador,3204126250,LLAMADA,NaN
1,2230,ILOCALIZABLES,C2C_2,N/D,0,Contestador,3204126250,LLAMADA,ALTA
2,2100,NAN,REINYECCION WHATSAPPS1,NAN,0,NaN,3057384654,CHAT,NaN
3,2100,NAN,REINYECCION WHATSAPPS1,NAN,0,NaN,3057384654,CHAT,NaN
4,2100,NAN,REINYECCION WHATSAPPS1,NAN,0,NaN,3057384654,CHAT,BAJA


In [254]:
df_criterios_telefono.head()

,0,1,2,3,4,5,6,7,8
0,LineaIn018000,LLAMADA,NaN,TIPIFICACIONES CHAT,NaN,NaN,NaN,TIPIFICACIONES LLAMADA,NaN
1,WhatsappS1,CHAT,NaN,SIN OFERTAMIENTO,TENER EN CUENTA,NaN,NaN,SIN COBERTURA,NO TENER EN CUENTA
2,C2C_2,LLAMADA,NaN,DIRECCION REQUIERE ESCALAMIENTO,TENER EN CUENTA,NaN,NaN,Cuelga en espera,TENER EN CUENTA
3,Linea_Inbound_linkedin,LLAMADA,NaN,CON OFERTAMIENTO,TENER EN CUENTA,NaN,NaN,NO VOLVER A LLAMAR,NO TENER EN CUENTA
4,Movil In,LLAMADA,NaN,NO ACEPTA NO INDICA RAZON,TENER EN CUENTA,NaN,NaN,LLAMADA NO SERIA,NO TENER EN CUENTA


In [255]:
# CREAR COLUMNA TIPIFICACION CON EL RANGO DE TELEFONOS

df_criterios_telefono = df_criterios_telefono.iloc[:125, 7:9]

df_criterios_telefono.columns = [
    "Nivel3",        # Columna H → CLAVE
    "TIPIFICACION"]  # Columna I → RESULTADO BUSCARV (2ª columna)


# Normalización para df2 (TELEFONO)
df2["Nivel3"] = df2["Nivel3"].astype(str).str.strip().str.upper()
df_criterios_telefono["Nivel3"] = df_criterios_telefono["Nivel3"].astype(str).str.strip().str.upper()

# Merge para df2
df2 = df2.merge(
    df_criterios_telefono[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)

In [256]:
#SE CAMBIA EL SEET ID POR EL DE AGENCIA COMPETITIVA

#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "10osDxoOJTMcaNbfJfJ7x8AhCvpqzeCtCImRwQ2k0Sbg"
sheet_name = "CONSOLIDADO_VOZ"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"


# 1) Leer Google Sheets (✓ correcto)
df_CONSOLIDADO_VOZ_agencia_competitiva = pd.read_csv(
    url,
    dtype=str
    #usecols=["TELEFONOS"]
)
df_CONSOLIDADO_VOZ_agencia_competitiva = df_CONSOLIDADO_VOZ_agencia_competitiva.rename(
    columns={df_CONSOLIDADO_VOZ_agencia_competitiva.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)

# 2) Normalizar (agregar esta parte que faltaba)
df2["TELEFONO2"] = df2["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_VOZ_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_VOZ_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()
)

df_CONSOLIDADO_VOZ_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_VOZ_agencia_competitiva["TELEFONOS_GOOGLE_SHEETS"]
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace("-", "", regex=False)
)

# 3) Merge CORREGIDO (lookup como BUSCARV)
df2 = df2.merge(
    df_CONSOLIDADO_VOZ_agencia_competitiva.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS con los teléfonos encontrados
df2["YAREINYECTADOS"] = df2["TELEFONOS_GOOGLE_SHEETS"]
df2.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)



In [257]:
df_CONSOLIDADO_VOZ_agencia_competitiva.shape

(1668, 3)

In [258]:
df2.sample(10)

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA,FUENTE,TIPIFICACION,FECHA,Unnamed: 2,YAREINYECTADOS
17,1400,NO VENTA,C2C_2,CPREMARKETINGHOGARES,0,NO HAY COBERTURA OTRA CIUDAD,3213394736,LLAMADA,ALTA,NO TENER EN CUENTA,NaN,NaN,NaN
690,1100,NO VENTA,WHATSAPPS1,NAN,0,CLIENTE YA CUENTA CON SERVICI ETB,3002761315,CHAT,NaN,NO TENER EN CUENTA,NaN,NaN,NaN
422,1530,NAN,WHATSAPPS1,NAN,0,NAN,3134453737,CHAT,BAJA,NO TENER EN CUENTA,NaN,NaN,NaN
897,1530,EN PROCESO,ETB INBOUND,GMBOGOTAFR,0,CUELGA EN ESPERA,3183090207,LLAMADA,ALTA,TENER EN CUENTA,07/01/2026,ALTA,3183090207
291,830,NO VENTA,WHATSAPPS1,NAN,0,NO HAY COBERTURA OTRA CIUDAD,3125264620,CHAT,NaN,NO TENER EN CUENTA,NaN,NaN,NaN
547,1200,EN PROCESO,C2C_2,ETBINTERNETEX,0,PROGRAMADO,3042423475,LLAMADA,ALTA,TENER EN CUENTA,06/01/2026,ALTA,3042423475
733,1030,NO VENTA,INBOUND_LOWCOST,GENERICOINTERNETAMPLIA,0,NO HAY COBERTURA OTRA CIUDAD,3005732873,LLAMADA,BAJA,NO TENER EN CUENTA,NaN,NaN,NaN
489,700,NO VENTA,C2C_2,CPPIEZASDISRUPTIVAS,0,CLIENTE YA CUENTA CON SERVICIOS ETB,3103705087,LLAMADA,BAJA,NO TENER EN CUENTA,NaN,NaN,NaN
637,2130,ILOCALIZABLES,C2C_2,N/D,0,CONTESTADOR,3219692874,LLAMADA,NaN,TENER EN CUENTA,07/01/2026,ALTA,3219692874
1178,800,NAN,C2C_2,CPREMARKETINGHOGARES,0,NAN,3212106895,LLAMADA,ALTA,NO TENER EN CUENTA,NaN,NaN,NaN


In [ ]:
df_llamada_competitiva = df2.copy()
df_llamada_competitiva = df_llamada_competitiva[df_llamada_competitiva['CATEGORIA'] == 'LLAMADA']  # ← CRÍTICO
df_llamada_competitiva['FUENTE'] = df_llamada_competitiva['FUENTE'].fillna('BAJA')
df_llamada_competitiva = df_llamada_competitiva[df_llamada_competitiva['TIPIFICACION'] == 'TENER EN CUENTA']
df_llamada_competitiva = df_llamada_competitiva[~df_llamada_competitiva['YAREINYECTADOS'].apply(pd.to_numeric, errors='coerce').notna()]

df_llamada_competitiva = df_llamada_competitiva.drop_duplicates(subset=["TELEFONO2", "CATEGORIA"], keep="first")

df_llamada_competitiva = df_llamada_competitiva[['TELEFONO2', 'FUENTE']]
df_llamada_competitiva["TELEFONO2"] = pd.to_numeric(df_llamada_competitiva["TELEFONO2"], errors="coerce")
df_llamada_competitiva = df_llamada_competitiva[(df_llamada_competitiva["TELEFONO2"] >= 3000000000) & (df_llamada_competitiva["TELEFONO2"] <= 3999999999)]

df_llamada_competitiva = df_llamada_competitiva.sort_values(by='TELEFONO2')
df_llamada_competitiva.rename(columns={'TELEFONO2': 'TELEFONO'}, inplace=True)

In [260]:

print(df_llamada_competitiva.shape)

(124, 2)


In [261]:
# Fecha: dd_mm_yyyy
fecha = datetime.now().strftime('%d_%m_%Y')

# CAMBIAR 0.1 → 01 (sin punto)
nombre_archivo = f"automatizado_0.1_{fecha}_VOZ_AGENCIA_COMPETITIVA.xlsx"

ruta_completa = f"C:/Users/sgarcial/Documents/ETB/Agencia competitiva/LLAMADAS/{nombre_archivo}"

df_llamada_competitiva.to_excel(ruta_completa, index=False)
print(f"✅ Guardado: {nombre_archivo}")

✅ Guardado: automatizado_0.1_08_01_2026_VOZ_AGENCIA_COMPETITIVA.xlsx
